In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython import display 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/steam-200k.csv',header=None)

In [ ]:
# use at jupyter notebook
data = pd.read_csv('data/steam-200k.csv',header=None)

In [5]:
# delete unnecessary data
data = data.drop([4],axis=1)
data.columns = ['userid','gametitle','behaviour','value']

# group by purchase and play
behv = data.groupby('behaviour')

purc = behv.get_group('purchase')
ply = behv.get_group('play')

# Data processing

After we split the data into Purchasing and Playing part, now it is time for us to do data processing

In [6]:
# counts total number of games being purchased by the users
purc['counts'] = purc.groupby('gametitle')['gametitle'].transform('size')

# use MinMaxScaler to scale the number of purchases into ratings, in scale of 1-5
from sklearn.preprocessing import MinMaxScaler

ori_purc_count = purc['counts'].values.reshape(-1,1)
scalar = MinMaxScaler(feature_range=(1,5))
purc_count_trans = scalar.fit_transform(ori_purc_count)

purc['counts_trans'] = purc_count_trans

# round up functions here
import math

def roundup(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

# round the ratings to the nearest integer
purc['counts_trans'] = purc.counts_trans.apply(lambda n:roundup(n))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [7]:
# next we install surprise library
!pip install surprise

     |████████████████████████████████| 11.8 MB 3.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619394 sha256=36a19e32a22812053912eb3679a45b6eca110988c36f4c33b489173b2701460a
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [8]:
from surprise import Reader, Dataset
reader = Reader(rating_scale = (1.0,5.0))
data_purc = Dataset.load_from_df(purc[['userid','gametitle','counts_trans']],reader)

Model Training

Now we training the purchasing data

In [9]:
from surprise.model_selection import train_test_split

# set train test as 75% 25%
trainset, testset = train_test_split(data_purc, test_size=0.25)

# train the model
from surprise import SVD, accuracy
algo = SVD(n_epochs=50, n_factors=50, verbose=True)
algo.fit(trainset)

# validate the model accuracy
from surprise.model_selection import cross_validate
cross_validate(algo, data_purc, measures=['RMSE','MAE'], cv=5, verbose=0)

Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing

{'fit_time': (10.149754047393799,
  10.31432056427002,
  10.193566799163818,
  10.186446189880371,
  10.389919519424438),
 'test_mae': array([0.03307733, 0.03318721, 0.03327754, 0.03366263, 0.03362549]),
 'test_rmse': array([0.05510479, 0.05513828, 0.05499408, 0.05582922, 0.05573016]),
 'test_time': (0.3401031494140625,
  0.31637144088745117,
  0.3757472038269043,
  0.22873568534851074,
  0.2238445281982422)}

Make predictions on the purchasing testset

In [10]:
predictions = algo.test(testset)

Model Evaluation on the purchasing testset

In [11]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 0.0456


0.04557610670580696

To recommend next game based on user inputs

In [12]:
from collections import defaultdict

def get_all_predictions(predictions):
  # first map the predictions to each user
  top_n = defaultdict(list)
  for uid, iid, true_r, est, _ in predictions:
    top_n[uid].append((iid,est))

  # then sort the predictions for each user
  for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
  return top_n

all_pred_purc = get_all_predictions(predictions)

In [13]:
# filter userid where they have purchased at least 4 items
new_all_pred_purc = dict(filter(lambda elem: len(elem[1]) >= 4 , all_pred_purc.items()))

In [14]:
# prompt user asking what did they buy previously and return 5 next recommendations (with ratings)
game_name = input('What did you buy? ')

# from users who purchased at least 4 items, sort according to ratings
n = 4
for uid, user_ratings in new_all_pred_purc.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    new_all_pred_purc[uid] = user_ratings[:n]

# convert to DataFrame and transpose 
tmp = pd.DataFrame.from_dict(new_all_pred_purc)
tmp_transpose = tmp.transpose()

# define get_predictions function to find out the ratings of a certain user
def get_predictions(user_id):
    results = tmp_transpose.loc[user_id]
    return results

# now from the game that user input, find out which userid purchased the same game as his/hers
similar_user = tmp_transpose.apply(lambda row: row.astype(str).str.contains(str(game_name)).any(), axis=1)
similar_user_idx = similar_user[similar_user == True].index

# find out highest rating of these users, store in a list
s_user_item = []
for i in similar_user_idx:
  s_user_item.append(get_predictions(i))

# the below changes to a list of tuples
comb_tuple = []
for sub in s_user_item:
  for sub2 in sub:
    comb_tuple.append(sub2)

# define a function to sort the tuple according to the ratings, and only keep the highest rating for each game
def sort_tuple(tup):
  tup.sort(key = lambda x: x[1], reverse = True)
  return tup
comb_tuple_sorted = sort_tuple(comb_tuple)

# now keep the highest rating only for each game, the rest will be removed
from functools import reduce
comb_tuple_nodup = reduce(lambda lu, i:i[0] in dict(lu).keys() and lu or lu+[i], comb_tuple_sorted, [])
# high_tuple excludes the game user  has already purchased if the machine recommends again
high_tuple = [x for x in comb_tuple_nodup if x[0] != game_name]

# get the top 5 items
top5_purc = high_tuple[:5]

What did you buy? Crysis


After preparing and training models for purchasing data, now let's repeat the same steps for playing data

In [15]:
# take the total sum of the hours played by each users, on each game
ply['Total Hrs'] = ply.groupby('gametitle', sort=False)["value"].transform('sum')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
# scale the number of hours played to range from 1-5

ori_hrs = ply['Total Hrs'].values.reshape(-1,1)
scalar = MinMaxScaler(feature_range=(1,5))
hrs_trans = scalar.fit_transform(ori_hrs)

ply['hrs_trans'] = hrs_trans

# roundup the ratings to the nearest integer
ply['hrs_trans'] = ply.hrs_trans.apply(lambda n:roundup(n))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [19]:
# we revisit the surprise library
reader = Reader(rating_scale = (1.0,5.0))
data_ply = Dataset.load_from_df(ply[['userid','gametitle','hrs_trans']],reader)

In [20]:
# set train test as 75% 25%
trainset, testset = train_test_split(data_ply, test_size=0.25)

# train the model
from surprise import SVD, accuracy
algo = SVD(n_epochs=50, n_factors=50, verbose=True)
algo.fit(trainset)

# validate the model accuracy
from surprise.model_selection import cross_validate
cross_validate(algo, data_ply, measures=['RMSE','MAE'], cv=5, verbose=0)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

{'fit_time': (6.09004282951355,
  6.196782350540161,
  5.8806540966033936,
  6.34937310218811,
  5.9640421867370605),
 'test_mae': array([0.04410841, 0.04460544, 0.04410527, 0.04309877, 0.04455485]),
 'test_rmse': array([0.07183196, 0.07220988, 0.07151305, 0.07010363, 0.07194412]),
 'test_time': (0.15743565559387207,
  0.10882806777954102,
  0.18155932426452637,
  0.11167693138122559,
  0.14271140098571777)}

In [21]:
# Make predictions based on the trained model
predictions_ply = algo.test(testset)

from surprise import accuracy
accuracy.rmse(predictions_ply)

RMSE: 0.0549


0.05488868425450278

In [25]:
# get all the predictions for hours of game played
from collections import defaultdict

def get_all_predictions_ply(predictions):
  # first map the predictions to each user
  top_n = defaultdict(list)
  for uid, iid, true_r, est, _ in predictions:
    total_hrs_ply = ply.loc[ply['gametitle'] == iid, 'Total Hrs'].iloc[0]
    top_n[uid].append((iid,est,total_hrs_ply))

  # then sort the predictions for each user
  for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[2], reverse=True)
  return top_n

all_pred_ply = get_all_predictions_ply(predictions_ply)

In [39]:
new_all_pred_ply

{196595568: [("Garry's Mod", 1.0, 49725.300000000025),
  ('Borderlands 2', 1.0186504108732526, 22667.899999999994),
  ('Borderlands The Pre-Sequel', 1.0397410976411252, 2852.1),
  ('The Escapists', 1.034041239106983, 480.9999999999999)],
 32126281: [('Total War ROME II - Emperor Edition', 1.0, 20892.499999999996),
  ('Fallout 4', 1.013702320523621, 10762.800000000003),
  ('Napoleon Total War', 1.0, 9966.000000000004),
  ('Rocket League', 1.0, 7827.200000000002)],
 73835640: [("Sid Meier's Civilization V", 1.0, 99821.29999999999),
  ('Terraria', 1.0, 29951.799999999985),
  ('Torchlight II', 1.0, 6891.9),
  ('Dungeon Defenders', 1.0050806382602486, 3816.2000000000007)],
 53875128: [('Left 4 Dead 2', 1.0, 33596.69999999999),
  ('Portal 2', 1.0, 9117.099999999997),
  ('Half-Life 2', 1.0, 4260.3),
  ('Dishonored', 1.0, 2842.2)],
 124443679: [('Call of Duty Modern Warfare 2 - Multiplayer',
   1.0,
   42009.90000000001),
  ('PAYDAY 2', 1.0025357438496594, 19613.799999999996),
  ('Call of Duty

In [34]:
# filter userid where they have played at least 4 games
new_all_pred_ply = dict(filter(lambda elem: len(elem[1]) >= 4 , all_pred_ply.items()))

# from users who played at least 4 games, sort according to hours played
n = 4
for uid, user_ratings in new_all_pred_ply.items():
    user_ratings.sort(key=lambda x: x[2], reverse=True)
    new_all_pred_ply[uid] = user_ratings[:n]

# convert to DataFrame and transpose 
tmp_ply = pd.DataFrame.from_dict(new_all_pred_ply)
tmp_transpose_ply = tmp_ply.transpose()

# define get_predictions function to find out the ratings of a certain user
def get_predictions(user_id):
    results = tmp_transpose_ply.loc[user_id]
    return results

# now from the game that user input, find out which userid played the same game as his/hers
similar_user_ply = tmp_transpose_ply.apply(lambda row: row.astype(str).str.contains(str(game_name)).any(), axis=1)
similar_user_ply_idx = similar_user_ply[similar_user_ply == True].index

# find out top N highest hours played of these users, store in a list
s_user_item_ply = []
for i in similar_user_ply_idx:
  s_user_item_ply.append(get_predictions(i))

# the below changes to a list of tuples
comb_tuple_ply = []
for sub in s_user_item_ply:
  for sub2 in sub:
    comb_tuple_ply.append(sub2)

# define a function to sort the tuple according to the hours played
def sort_tuple(tup):
  tup.sort(key = lambda x: x[2], reverse = True)
  return tup
comb_tuple_ply_sorted = sort_tuple(comb_tuple_ply)

# reduce tuple dimension from 3 to 2
comb_tuple2_ply_sorted = []
for tu in comb_tuple_ply_sorted:
  comb_tuple2_ply_sorted.append((tu[0],tu[2]))

# now keep the highest rating only for each game, the rest will be removed
from functools import reduce
comb_tuple_ply_nodup = reduce(lambda lu, i:i[0] in dict(lu).keys() and lu or lu+[i], comb_tuple2_ply_sorted, [])

# high_tuple excludes the game user  has already played if the machine recommends again
high_tuple_ply = [x for x in comb_tuple_ply_nodup if x[0] != game_name]

# get the top 5 items
top5_ply = high_tuple_ply[:5]

# now print the outputs
print('Thanks for purchasing', game_name, '!\n')
print('You may like these (popularity from 1(min) to 5(max):)')
for x in top5_purc:
  print(x[0], ' (popularity:', format(x[1],'.1f'), ')')

# now print the outputs
print('\nYou may consider these games which were played the most:')
for x in top5_ply:
  ttl_hrs_ply = ply.loc[ply['gametitle'] == x[0], 'Total Hrs'].iloc[0]
  print(x[0], ' (Total Hours Played:', format(ttl_hrs_ply,'.1f'), ')')

Thanks for purchasing Crysis !

You may like these (popularity from 1(min) to 5(max):)
Dota 2  (popularity: 4.8 )
Team Fortress 2  (popularity: 2.9 )
Half-Life 2  (popularity: 2.0 )
Garry's Mod  (popularity: 2.0 )
The Elder Scrolls V Skyrim  (popularity: 2.0 )

You may consider these games which were played the most:
Unturned  (Total Hours Played: 16096.4 )
Portal 2  (Total Hours Played: 9117.1 )
Dungeon Defenders  (Total Hours Played: 3816.2 )
Divinity Original Sin  (Total Hours Played: 2380.1 )
Mortal Kombat Komplete Edition  (Total Hours Played: 1324.3 )
